# Building your vanilla RNN - Step by Step

Welcome to Course 5's first assignment! In this assignment, you will implement key components of a Recurrent Neural Network in numpy.

Recurrent Neural Networks (RNN) are very effective for Natural Language Processing and other sequence tasks because they have "memory". They can read inputs $x^{\langle t \rangle}$ (such as words) one at a time, and remember some information/context through the hidden layer activations that get passed from one time-step to the next. This allows a unidirectional RNN to take information from the past to process later inputs. A bidirectional RNN can take context from both the past and the future. 

**Notation**:
- Superscript $[l]$ denotes an object associated with the $l^{th}$ layer. 

- Superscript $(i)$ denotes an object associated with the $i^{th}$ example. 

- Superscript $\langle t \rangle$ denotes an object at the $t^{th}$ time-step. 
    
- **Sub**script $i$ denotes the $i^{th}$ entry of a vector.

Example:  
- $a^{(2)[3]<4>}_5$ denotes the activation of the 2nd training example (2), 3rd layer [3], 4th time step <4>, and 5th entry in the vector.

#### Pre-requisites
* We assume that you are already familiar with `numpy`.  
* To refresh your knowledge of numpy, you can review course 1 of this specialization "Neural Networks and Deep Learning".  
    * Specifically, review the week 2 assignment ["Python Basics with numpy (optional)"](https://www.coursera.org/learn/neural-networks-deep-learning/item/Zh0CU).
    
    
Let's first import all the packages that you will need during this assignment.

In [1]:
import numpy as np
from rnn_utils import *

## 1. Forward propagation for the basic RNN

- Later this week, you will generate music using an RNN. 
- The basic RNN that you will implement has the structure below. In this example, $T_x = T_y$. 

<img src="images/RNN.png" style="width:500;height:300px;">
<caption><center><b>Figure 1</b>: Basic RNN model </center></caption>

### Dimensions of input $x$

#### 1. Input with $n_x$ number of units
- For a single timestep of a single input example, $x^{(i) \langle t \rangle }$ is a **1-dimensional** input vector.
- Using language as an example, a language with a 5000 word vocabulary could be one-hot encoded into a vector that has 5000 units.  So $x^{(i)\langle t \rangle}$ would have the shape (5000,).  
- We will use the notation $n_x$ to denote the number of units in a single timestep of a single training example.

#### 2. Time steps of size $T_{x}$
- A recurrent neural network has multiple time steps, which we will index with $t$.
- In the lessons, we saw a single training example $x^{(i)}$ consists of multiple time steps $T_x$.  
  For example, if there are 10 time steps, $T_{x} = 10$
  
#### 3. Batches of size $m$
- Let's say we have mini-batches, each with 20 training examples.  
- To benefit from vectorization, we will stack 20 columns of $x^{(i)}$ examples.
- For example, this tensor has the shape (5000, 20, 10). 
- We will use $m$ to denote the number of training examples.  
- So the shape of a mini-batch is $(n_x, m, T_x)$  

#### 4. 3D Tensor of shape $(n_{x},m,T_{x})$
- The 3-dimensional tensor $x$ of shape $(n_x, m, T_x)$ represents the input $x$ that is fed into the RNN.

#### 5. Taking a 2D slice for each time step: $x^{\langle t \rangle}$
- At each time step, we will use a mini-batch of training examples (not just a single example).
- So, for each time step $t$, we will use a 2D slice of shape $(n_x, m)$.
- We are referring to this 2D slice as $x^{\langle t \rangle}$.  
  The variable name in the code is `xt`.
  
<hr />  

### Definition of hidden state $a$

- The activation $a^{\langle t \rangle}$ that is passed to the RNN from one time step to another is called a *hidden state*.

### Dimensions of hidden state $a$

- Similar to the input tensor $x$, the hidden state for a single training example is a vector of length $n_{a}$.
- If we include a mini-batch of $m$ training examples, the shape of a mini-batch is $(n_{a}, m)$.
- When we include the time step dimension, the shape of the hidden state is $(n_{a}, m, T_x)$
- We will loop through the time steps with index $t$, and work with a 2D slice of the 3D tensor.  
- We will refer to this 2D slice as $a^{\langle t \rangle}$. 
- In the code, the variable names we use are either `a_prev` or `a_next`, depending on the function that's being implemented.
- The shape of this 2D slice is $(n_{a}, m)$

<hr />

### Dimensions of prediction $\hat{y}$
- Similar to the inputs and hidden states, $\hat{y}$ is a 3D tensor of shape $(n_{y},\ m,\ T_{y})$.
    * $n_{y}$: number of units in the vector representing the prediction.
    * $m$: number of examples in a mini-batch.
    * $T_{y}$: number of time steps in the prediction.


- For a single time step $t$, a 2D slice $\hat{y}^{\langle t \rangle}$ has shape $(n_{y}, m)$.


- In the code, the variable names are:
  - `y_pred` stands for $\hat{y}$ 
  - `yt_pred` stands for $\hat{y}^{\langle t \rangle}$
    
<hr />



Here's how you can implement an RNN: 

**Steps**:
1. Implement the calculations needed for one time-step of the RNN.
2. Implement a loop over $T_x$ time-steps in order to process all the inputs, one at a time. 

## 1.1. RNN cell

A RNN can be seen as the repeated use of a single cell.  
You are first going to implement the computations for a single time-step. The following figure describes the operations for a single time-step of an RNN cell. 

<img src="images/rnn_step_forward_figure2_v3a.png" style="width:700px;height:300px;">
<caption><center><b>Figure 2</b>: Basic RNN cell.<br /> Takes as input $x^{\langle t \rangle}$ (current input) and $a^{\langle t - 1\rangle}$ (prev. hidden state containing info. from the past) and outputs $a^{\langle t \rangle}$ <br />which is given to the next RNN cell and also used to predict $\hat{y}^{\langle t \rangle}$ </center></caption>

#### rnn cell versus rnn_cell_forward
- Note that an RNN cell outputs the hidden state $a^{\langle t \rangle}$.  
  The rnn cell is shown in the figure as the *inner box which has solid lines*.  


- The function that we will implement, `rnn_cell_forward`, also calculates the prediction $\hat{y}^{\langle t \rangle}$  
  The rnn_cell_forward is shown in the figure as the *outer box that has dashed lines*.
  

**Exercise**: Implement the RNN-cell described in Figure (2).

**Instructions**:
1. Compute the hidden state with tanh activation: $a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$.
2. Using your new hidden state $a^{\langle t \rangle}$, compute the prediction $\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$. We provided the function `softmax`.
3. Store $(a^{\langle t \rangle}, a^{\langle t-1 \rangle}, x^{\langle t \rangle}, parameters)$ in a `cache`.
4. Return $a^{\langle t \rangle}$ , $\hat{y}^{\langle t \rangle}$ and `cache`

#### Additional Hints
* [numpy.tanh](https://www.google.com/search?q=numpy+tanh&rlz=1C5CHFA_enUS854US855&oq=numpy+tanh&aqs=chrome..69i57j0l5.1340j0j7&sourceid=chrome&ie=UTF-8)
* We've created a `softmax` function that you can use.  It is located in the file 'rnn_utils.py' and has been imported.
* For matrix multiplication, use [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html)

In [2]:
# GRADED FUNCTION: rnn_cell_forward

def rnn_cell_forward(xt, a_prev, params):
    """
    Implements a single forward step of the RNN-cell as described in Figure (2)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    params -- python dictionary containing:
                Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                ba --  Bias, numpy array of shape (n_a, 1)
                by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, a_prev, xt, parameters)
    """
    
    # Retrieve parameters from "parameters"
    Wax, Waa, Wya = params["Wax"], params["Waa"], params["Wya"]
    ba, by = params["ba"], params["by"]
    
    ### CODE HERE:
    a_next = np.tanh(Wax.dot(xt) + Waa.dot(a_prev) + ba) # 1. Calc next activ. state using formula above
    yt_pred = softmax(Wya.dot(a_next) + by)              # 2. Calc output of the curr. cell using formula above
    cache = (a_next, a_prev, xt, params)                 # 3. store values needed for backward propagation
    
    return a_next, yt_pred, cache

In [3]:
np.random.seed(1)
xt_tmp = np.random.randn(3,10)
a_prev_tmp = np.random.randn(5,10)

params_tmp = {}
params_tmp['Waa'] = np.random.randn(5,5)
params_tmp['Wax'] = np.random.randn(5,3)
params_tmp['Wya'] = np.random.randn(2,5)
params_tmp['ba'] = np.random.randn(5,1)
params_tmp['by'] = np.random.randn(2,1)

a_next_tmp, yt_pred_tmp, cache_tmp = rnn_cell_forward(xt_tmp, a_prev_tmp, params_tmp)
print("a_next[4] = \n", a_next_tmp[4])
print("a_next.shape = \n", a_next_tmp.shape)
print("yt_pred[1] =\n", yt_pred_tmp[1])
print("yt_pred.shape = \n", yt_pred_tmp.shape)

a_next[4] = 
 [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape = 
 (5, 10)
yt_pred[1] =
 [0.9888161  0.01682021 0.21140899 0.36817467 0.98988387 0.88945212
 0.36920224 0.9966312  0.9982559  0.17746526]
yt_pred.shape = 
 (2, 10)


**Expected Output**: 
```Python
a_next[4] = 
 [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape = 
 (5, 10)
yt_pred[1] =
 [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
yt_pred.shape = 
 (2, 10)

```

## 1.2. RNN forward pass 

- A recurrent neural network (RNN) is a repetition of the RNN cell that you have just built. 
  - If your input sequence of data is 10 time steps long, then you will re-use the RNN cell 10 times. 
  

- Each cell takes two inputs at each time step:
  - $a^{\langle t-1 \rangle}$: The hidden state from the previous cell.
  - $x^{\langle t \rangle}$: The current time-step's input data.


- It has two outputs at each time step:
  - A hidden state ($a^{\langle t \rangle}$)
  - A prediction ($y^{\langle t \rangle}$)


- The weights and biases $(W_{aa}, b_{a}, W_{ax}, b_{x})$ are re-used at each time step. 
  - They are maintained between calls to rnn_cell_forward in the 'parameters' dictionary.


<img src="images/rnn_forward_sequence_figure3_v3a.png" style="width:800px;height:180px;">
<caption><center><b>Figure 3</b>: Basic RNN. The input sequence $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$  is carried over $T_x$ time steps. <br />The network outputs $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$. </center></caption>

**Exercise**: Code the forward propagation of the RNN described in Figure (3).

**Instructions**:
- Create a 3D array of zeros, $a$ of shape $(n_{a}, m, T_{x})$ that will store all the hidden states computed by the RNN.

- Create a 3D array of zeros, $\hat{y}$, of shape $(n_{y}, m, T_{x})$ that will store the predictions.  
  - Note that in this case, $T_{y} = T_{x}$ (the prediction and input have the same number of time steps).

- Initialize the 2D hidden state `a_next` by setting it equal to the initial hidden state, $a_{0}$.

- At each time step $t$:
  - Get $x^{\langle t \rangle}$, which is a 2D slice of $x$ for a single time step $t$.
    - $x^{\langle t \rangle}$ has shape $(n_{x}, m)$, while $x$ has shape $(n_{x}, m, T_{x})$
  - Update the 2D hidden state $a^{\langle t \rangle}$ (denoted `a_next`), the prediction $\hat{y}^{\langle t \rangle}$ and the cache by running `rnn_cell_forward`.
    - $a^{\langle t \rangle}$ has shape $(n_{a}, m)$
  - Store the 2D hidden state in the 3D tensor $a$, at the $t^{th}$ position.
    - $a$ has shape $(n_{a}, m, T_{x})$
  - Store the 2D $\hat{y}^{\langle t \rangle}$ prediction (variable name `yt_pred`) in the 3D tensor $\hat{y}_{pred}$ at the $t^{th}$ position.
    - $\hat{y}^{\langle t \rangle}$ has shape $(n_{y}, m)$, while $\hat{y}$ has shape $(n_{y}, m, T_x)$
  - Append the cache to the list of caches.

- Return the 3D tensor $a$ and $\hat{y}$, as well as the list of caches.


In [4]:
# GRADED FUNCTION: rnn_forward

def rnn_forward(x, a0, params):
    """
    Implement the forward propagation of the recurrent neural network described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    params -- python dictionary containing:
                Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                ba --  Bias numpy array of shape (n_a, 1)
                by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)

    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of caches, x)
    """
    
    caches = []                                              # Initialize "caches" 
    (n_x, m, T_x), (n_y, n_a) = x.shape, params["Wya"].shape # Retrieve dims from shapes of x and params["Wya"]
    
    ### CODE HERE:
    a, y_pred = np.zeros((n_a, m, T_x)), np.zeros((n_y, m, T_x)) # Initialize "a" and "y_pred" with zeros
    a_next = a0                                                  # Initialize a_next
     
    for t in range(T_x):                                         # Loop over all time-steps of the input 'x'
        
        xt = x[:, :, t]                                          # Update next hidden state
        a_next, yt_pred, cache = rnn_cell_forward(xt, a_next, params) # Calc the prediction, get the cache
        
        a[:, :, t] = a_next                                      # Save the value of new "next" hidden state
        y_pred[:, :, t] = yt_pred                                # Save the value of prediction in y
        caches.append(cache)
    #
    caches = (caches, x)                                         # Store values needed for backward propagation
    
    return a, y_pred, caches

In [5]:
np.random.seed(1)
x_tmp = np.random.randn(3,10,4)
a0_tmp = np.random.randn(5,10)
params_tmp = {}
params_tmp['Waa'] = np.random.randn(5,5)
params_tmp['Wax'] = np.random.randn(5,3)
params_tmp['Wya'] = np.random.randn(2,5)
params_tmp['ba'] = np.random.randn(5,1)
params_tmp['by'] = np.random.randn(2,1)

a_tmp, y_pred_tmp, caches_tmp = rnn_forward(x_tmp, a0_tmp, params_tmp)
print("a[4][1] = \n", a_tmp[4][1])
print("a.shape = ", a_tmp.shape)
print("y_pred[1][3] =\n", y_pred_tmp[1][3])
print("y_pred.shape = ", y_pred_tmp.shape)
print("caches[1][1][3] =\n", caches_tmp[1][1][3])
print("len(caches) = ", len(caches_tmp), " / len(caches[0]) = ", len(caches_tmp[0]))

a[4][1] = 
 [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape =  (5, 10, 4)
y_pred[1][3] =
 [0.79560373 0.86224861 0.11118257 0.81515947]
y_pred.shape =  (2, 10, 4)
caches[1][1][3] =
 [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) =  2  / len(caches[0]) =  4


**Expected Output**:

```Python
a[4][1] = 
 [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape = (5, 10, 4)
y_pred[1][3] =
 [ 0.79560373  0.86224861  0.11118257  0.81515947] 
y_pred.shape = (2, 10, 4)
caches[1][1][3] =
 [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) = 2 / len(caches[0]) = 4
```

Congratulations! You've successfully built the forward propagation of a RNN from scratch. 

#### Situations when this RNN will perform better:
- This will work well enough for some applications, but it suffers from the vanishing gradient problem. 
- The RNN works best when each output $\hat{y}^{\langle t \rangle}$ can be estimated using *local context*.  
- *Local context* refers to information that is close to the prediction's time step $t$.
- More formally, *local context* refers to inputs $x^{\langle t' \rangle}$ and predictions $\hat{y}^{\langle t \rangle}$ where $t'$ is close to $t$.

In the next part, you will build a more complex LSTM model, which is better at addressing vanishing gradients.  
The LSTM will be better able to remember a piece of information and keep it saved for many timesteps. 
                                                      
<hr />                                                      

## 2. Backpropagation for basic RNN

In modern deep learning frameworks, you only have to implement the forward pass, and the framework takes care of the backward pass, so most deep learning engineers do not need to bother with the details of the backward pass. If however you are an expert in calculus and want to see the details of backprop in RNNs, you can work through this optional portion of the notebook. 

When in an earlier [course](https://www.coursera.org/learn/neural-networks-deep-learning/lecture/0VSHe/derivatives-with-a-computation-graph)  you implemented a simple (fully connected) neural network, you used backpropagation to compute the derivatives with respect to the cost to update the parameters. Similarly, in recurrent neural networks you can calculate the derivatives with respect to the cost in order to update the parameters. The backprop equations are quite complicated and we did not derive them in lecture. However, we will briefly present them below. 

*Note* that this notebook does NOT implement the backward path from the Loss 'J' backwards to 'a'. This would have included the dense layer and softmax which are a part of the forward path. This is assumed to be calculated elsewhere and the result passed to rnn_backward in 'da'. It is further assumed that loss has been adjusted for batch size (m) and division by the number of examples is not required here.


This section is optional and ungraded. It is more difficult and has fewer details regarding its implementation. This section only implements key elements of the full path.

### 2.1. Basic RNN  backward pass

We will start by computing the backward pass for the basic RNN-cell and then in the following sections, iterate through the cells.

<img src="images/rnn_backward_overview_3a_1.png" style="width:500;height:300px;"> <br>
<caption><center><b>Figure 6</b>: RNN-cell's backward pass. Just like in a fully-connected NN, the derivative of the cost function $J$ backpropagates through the time steps of the RNN by following the chain-rule from calculus. Internal to the cell, the chain-rule is also used to calculate $(\frac{\partial J}{\partial W_{ax}},\frac{\partial J}{\partial W_{aa}},\frac{\partial J}{\partial b})$ to update <br /> the parameters $(W_{ax}, W_{aa}, b_a)$. The operation can utilize the cached results from the forward path. </center></caption>

Recall from lecture, the shorthand for the partial derivative of cost relative to a variable is *dvariable*.  
For example, $\frac{\partial J}{\partial W_{ax}}$ is $dW_{ax}$ and in code: `dWax`. This will be used throughout the remaining sections.

<img src="images/rnn_cell_backward_3a_4.png" style="width:500;height:300px;"> <br>
<caption><center><b>Figure 7</b>: This implementation of rnn_cell_backward does not include the output dense layer and softmax which are included in rnn_cell_forward. <br />
$da_{next}$ is $\frac{\partial{J}}{\partial a^{\langle t \rangle}}$ and includes loss from previous stages and current stage output logic. This addition will be part of your implementation of rnn_backward.  </center></caption>

##### Equations
To compute the rnn_cell_backward you can utilize the following equations. It is a good exercise to derive them by hand.  
Here, $*$ denotes element-wise multiplication while the absence of a symbol indicates matrix multiplication.

$a^{\langle t \rangle} = \tanh(W_{ax} x^{\langle t \rangle} + W_{aa} a^{\langle t-1 \rangle} + b_{a})\tag{-}$
 
$\displaystyle \frac{\partial \tanh(x)} {\partial x} = 1 - \tanh^2(x) \tag{-}$
 
$\displaystyle  {dW_{ax}} = (da_{next} * (1-\tanh^2(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a})))\  x^{\langle t \rangle T} = (da_{next} * dz) \ x^{\langle t \rangle T} \tag{1}$

$\displaystyle dW_{aa} = da_{next} * (1-\tanh^2(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a}) )\  a^{\langle t-1 \rangle T} = (da_{next} * dz) \ a^{\langle t-1 \rangle T}\tag{2}$

$\displaystyle db_a = \sum_{batch} da_{next} * (1-\tanh^2(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a}))\tag{3} = \sum_{batch} (da_{next} * dz)$
 
$\displaystyle dx^{\langle t \rangle} = {W_{ax}}^T (da_{next} * ( 1-\tanh^2(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a}))) = {W_{ax}}^T (da_{next} * dz) \tag{4}$
  
$\displaystyle da_{prev} = {W_{aa}}^T (da_{next} * (1-\tanh^2(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a}))) = {W_{aa}}^T (da_{next} * dz)\tag{5}$

#### Implementing rnn_cell_backward
The results can be computed directly by implementing the equations above.  
However, the above can optionally be simplified by computing 'dz' and utlilizing the chain rule.  
This can be further simplified by noting that $\tanh(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a})$ was computed and saved in the forward pass. 

To calculate dba, the 'batch' above is a sum across the horizontal (axis= 1) axis. Note that you should use the keepdims = True option.

It may be worthwhile to review Course 1 [Derivatives with a computational graph](https://www.coursera.org/learn/neural-networks-deep-learning/lecture/0VSHe/derivatives-with-a-computation-graph)  through  [Backpropagation Intuition](https://www.coursera.org/learn/neural-networks-deep-learning/lecture/6dDj7/backpropagation-intuition-optional), which decompose the calculation into steps using the chain rule. Matrix vector derivatives are described [here](http://cs231n.stanford.edu/vecDerivs.pdf), though the equations above incorporate the required transformations.

Note rnn_cell_backward does __not__ include the calculation of loss from $y \langle t \rangle$, this is incorporated into the incoming da_next. This is a slight mismatch with rnn_cell_forward which includes a dense layer and softmax. 

Note: in the code:  
- $\displaystyle dx^{\langle t \rangle}$ is represented by dxt,   
- $\displaystyle d W_{ax}$ is represented by dWax,   
- $\displaystyle da_{prev}$ is represented by da_prev,    
- $\displaystyle dW_{aa}$ is represented by dWaa,   
- $\displaystyle db_{a}$ is represented by dba,   

dz is not derived above but can optionally be derived by students to simplify the repeated calculations.

In [6]:
def rnn_cell_backward(da_next, cache):
    """
    Implements the backward pass for the RNN-cell (single time-step).

    Arguments:
    da_next -- Gradient of loss with respect to next hidden state
    cache -- python dictionary containing useful values (output of rnn_cell_forward())

    Returns:
    gradients -- python dictionary containing:
                   dx -- Gradients of input data, of shape (n_x, m)
                   da_prev -- Gradients of previous hidden state, of shape (n_a, m)
                   dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                   dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                   dba -- Gradients of bias vector, of shape (n_a, 1)
    """    
    # Retrieve values from cache
    (a_next, a_prev, xt, params) = cache
    
    # Retrieve values from parameters
    Wax, Waa, Wya = params["Wax"], params["Waa"], params["Wya"]
    ba, by = params["ba"] = params["by"]

    ### CODE HERE:
    dz = (1 - a_next ** 2)                            # Compute the gradient of the loss with respect to z
      
    dxt  = Wax.T.dot(da_next * dz)                    # Compute grad wrt xt according to eq(4)
    dWax = (da_next * dz).dot(xt.T)                   # Compute grad wrt Wax according to eq(1)

    da_prev = Waa.T.dot(da_next * dz)                 # Compute grad wrt a_prev according to eq(5) 
    dWaa = (da_next * dz).dot(a_prev.T)               # Compute grad wrt Waa according to eq(2)

    dba = np.sum(da_next * dz, axis=1, keepdims=True) # Compute gradient wrt b, acc. to (3)

    # Store the gradients in a python dictionary
    gradients = {"dxt": dxt, "da_prev": da_prev, "dWax": dWax, 
                 "dWaa": dWaa, "dba": dba}
    
    return gradients

In [7]:
np.random.seed(1)
xt_tmp = np.random.randn(3, 10)
a_prev_tmp = np.random.randn(5, 10)

params_tmp = {}
params_tmp['Wax'] = np.random.randn(5, 3)
params_tmp['Waa'] = np.random.randn(5, 5)
params_tmp['Wya'] = np.random.randn(2, 5)
params_tmp['ba']  = np.random.randn(5, 1)
params_tmp['by']  = np.random.randn(2, 1)

a_next_tmp, yt_tmp, cache_tmp = rnn_cell_forward(xt_tmp, a_prev_tmp, params_tmp)
da_next_tmp = np.random.randn(5, 10)  # (5, 10)

gradients_tmp = rnn_cell_backward(da_next_tmp, cache_tmp)

print("gradients[\"dxt\"][1][2] =", gradients_tmp["dxt"][1][2])
print("gradients[\"dxt\"].shape =", gradients_tmp["dxt"].shape)
print("gradients[\"da_prev\"][2][3] =", gradients_tmp["da_prev"][2][3])
print("gradients[\"da_prev\"].shape =", gradients_tmp["da_prev"].shape)
print("gradients[\"dWax\"][3][1] =", gradients_tmp["dWax"][3][1])
print("gradients[\"dWax\"].shape =", gradients_tmp["dWax"].shape)
print("gradients[\"dWaa\"][1][2] =", gradients_tmp["dWaa"][1][2])
print("gradients[\"dWaa\"].shape =", gradients_tmp["dWaa"].shape)
print("gradients[\"dba\"][4] =", gradients_tmp["dba"][4])
print("gradients[\"dba\"].shape =", gradients_tmp["dba"].shape)

gradients["dxt"][1][2] = -1.3872130506020925
gradients["dxt"].shape = (3, 10)
gradients["da_prev"][2][3] = -0.15239949377395495
gradients["da_prev"].shape = (5, 10)
gradients["dWax"][3][1] = 0.4107728249354584
gradients["dWax"].shape = (5, 3)
gradients["dWaa"][1][2] = 1.1503450668497135
gradients["dWaa"].shape = (5, 5)
gradients["dba"][4] = [0.20023491]
gradients["dba"].shape = (5, 1)


**Expected Output**:

<table>
    <tr>
        <td>
            <b>gradients["dxt"][1][2]</b>=
        </td>
        <td>
           -1.3872130506
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dxt"].shape</b>=
        </td>
        <td>
           (3, 10)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["da_prev"][2][3]</b>=
        </td>
        <td>
           -0.152399493774
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["da_prev"].shape</b>=
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWax"][3][1]</b>=
        </td>
        <td>
           0.410772824935
        </td>
    </tr>
            <tr>
        <td>
            <b>gradients["dWax"].shape</b>=
        </td>
        <td>
           (5, 3)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWaa"][1][2]</b>= 
        </td>
        <td>
           1.15034506685
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWaa"].shape</b>=
        </td>
        <td>
           (5, 5)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dba"][4]</b>= 
        </td>
        <td>
           [ 0.20023491]
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dba"].shape</b>= 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

#### Backward pass through the RNN

Computing the gradients of the cost with respect to $a^{\langle t \rangle}$ at every time-step $t$ is useful because it is what helps the gradient backpropagate to the previous RNN-cell. To do so, you need to iterate through all the time steps starting at the end, and at each step: 
- you increment the overall $db_a$, $dW_{aa}$, $dW_{ax}$ and 
- you store $dx$.

**Instructions**:

Implement the `rnn_backward` function. Initialize the return variables with zeros first and then loop through all the time steps while calling the `rnn_cell_backward` at each time timestep, update the other variables accordingly.


* Note that this notebook does not implement the backward path from the Loss 'J' backwards to 'a'. 
    * This would have included the dense layer and softmax which are a part of the forward path. 
    * This is assumed to be calculated elsewhere and the result passed to rnn_backward in 'da'. 
    * You must combine this with the loss from the previous stages when calling rnn_cell_backward (see figure 7 above).
* It is further assumed that loss has been adjusted for batch size (m).
    * Therefore, division by the number of examples is not required here.

In [8]:
def rnn_backward(da, caches):
    """
    Implement the backward pass for a RNN over an entire sequence of input data.

    Arguments:
    da -- Upstream gradients of all hidden states, of shape (n_a, m, T_x)
    caches -- tuple containing information from the forward pass (rnn_forward)
    
    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradient w.r.t. the input data, numpy-array of shape (n_x, m, T_x)
                        da0 -- Gradient w.r.t the initial hidden state, numpy-array of shape (n_a, m)
                        dWax -- Gradient w.r.t the input's weight matrix, numpy-array of shape (n_a, n_x)
                        dWaa -- Gradient w.r.t the hidden state's weight matrix, numpy-arrayof shape (n_a, n_a)
                        dba -- Gradient w.r.t the bias, of shape (n_a, 1)
    """
        
    ### CODE HERE:
    
    # Retrieve values from the first cache (t=1) of caches
    (caches, x) = caches
    (a1, a0, x1, params) = caches[0] # at t=1
    
    # Retrieve dimensions from da's and x1's shapes
    (n_a, m, T_x), (n_x, m) = da.shape, x1.shape
    
    # initialize the gradients with the right sizes
    dx   = np.zeros(x.shape)
    dWax = np.zeros(params['Wax'].shape)  # np.zeros((n_x, m, Tx))
    dWaa = np.zeros(params['Waa'].shape)  # np.zeros((n_a, n_a))
    dba  =  np.zeros((n_a, 1))            # np.zeros(params['dba'].shape) ? 
    da0  = np.zeros((n_a, m))
    da_prevt = np.zeros((n_a, m))
    
    # Loop through all the time steps
    for t in reversed(range(T_x)):
        # Compute gradients at time step t. 
        # Remember to sum gradients from the output path (da) and the previous timesteps (da_prevt)
        gradients = rnn_cell_backward(da[:, :, t] + da_prevt, caches[t])
        
        # Retrieve derivatives from gradients (≈ 1 line)
        dxt, da_prevt, dWaxt, dWaat, dbat = gradients["dxt"], gradients["da_prev"], gradients["dWax"], \
            gradients["dWaa"], gradients["dba"]
        
        # Increment global derivatives w.r.t parameters by adding their derivative at time-step t
        dx[:, :, t] = dxt
        dWax += dWaxt
        dWaa += dWaat
        dba  += dbat
        
    # Set da0 to the gradient of a, which has been backpropagated through all time-steps
    da0 = da[:, :, t]
    
    # Store the gradients in a python dictionary
    gradients = {"dx": dx, "da0": da0, "dWax": dWax, "dWaa": dWaa,"dba": dba}
    
    return gradients

In [9]:
np.random.seed(1)
x_tmp = np.random.randn(3, 10, 4)
a0_tmp = np.random.randn(5, 10)

params_tmp = {}
params_tmp['Wax'] = np.random.randn(5,3)
params_tmp['Waa'] = np.random.randn(5,5)
params_tmp['Wya'] = np.random.randn(2,5)
params_tmp['ba'] = np.random.randn(5,1)
params_tmp['by'] = np.random.randn(2,1)

a_tmp, y_tmp, caches_tmp = rnn_forward(x_tmp, a0_tmp, params_tmp)
da_tmp = np.random.randn(5, 10, 4)

print(len(caches_tmp[0]))

gradients_tmp = rnn_backward(da_tmp, caches_tmp)

print("gradients[\"dx\"][1][2] =", gradients_tmp["dx"][1][2])
print("gradients[\"dx\"].shape =", gradients_tmp["dx"].shape)
print("gradients[\"da0\"][2][3] =", gradients_tmp["da0"][2][3])
print("gradients[\"da0\"].shape =", gradients_tmp["da0"].shape)
print("gradients[\"dWax\"][3][1] =", gradients_tmp["dWax"][3][1])
print("gradients[\"dWax\"].shape =", gradients_tmp["dWax"].shape)
print("gradients[\"dWaa\"][1][2] =", gradients_tmp["dWaa"][1][2])
print("gradients[\"dWaa\"].shape =", gradients_tmp["dWaa"].shape)
print("gradients[\"dba\"][4] =", gradients_tmp["dba"][4])
print("gradients[\"dba\"].shape =", gradients_tmp["dba"].shape)

4
gradients["dx"][1][2] = [-2.07101689 -0.59255627  0.02466855  0.01483317]
gradients["dx"].shape = (3, 10, 4)
gradients["da0"][2][3] = -0.8677686776235903
gradients["da0"].shape = (5, 10)
gradients["dWax"][3][1] = 11.264104496527777
gradients["dWax"].shape = (5, 3)
gradients["dWaa"][1][2] = 2.303333126579893
gradients["dWaa"].shape = (5, 5)
gradients["dba"][4] = [-0.74747722]
gradients["dba"].shape = (5, 1)


**Expected Output**:

<table>
    <tr>
        <td>
            <b>gradients["dx"][1][2]</b>=
        </td>
        <td>
           [-2.07101689 -0.59255627  0.02466855  0.01483317]
        </td>
    </tr>
        <tr>
        <td>
           <b>gradients["dx"].shape</b>=
        </td>
        <td>
           (3, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
           <b>gradients["da0"][2][3]</b>=
        </td>
        <td>
           -0.314942375127
        </td>
    </tr>
        <tr>
        <td>
           <b>gradients["da0"].shape</b>=
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
         <tr>
        <td>
           <b>gradients["dWax"][3][1]</b>=
        </td>
        <td>
           11.2641044965
        </td>
    </tr>
        <tr>
        <td>
           <b>gradients["dWax"].shape</b>=
        </td>
        <td>
           (5, 3)
        </td>
    </tr>
        <tr>
        <td>
           <b>gradients["dWaa"][1][2]</b>= 
        </td>
        <td>
           2.30333312658
        </td>
    </tr>
        <tr>
        <td>
           <b>gradients["dWaa"].shape</b>=
        </td>
        <td>
           (5, 5)
        </td>
    </tr>
        <tr>
        <td>
           <b>gradients["dba"][4]</b>= 
        </td>
        <td>
           [-0.74747722]
        </td>
    </tr>
        <tr>
        <td>
           <b>gradients["dba"].shape</b>= 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

### Congratulations !

Congratulations on completing this assignment. You now understand how recurrent neural networks work! 

Let's go on to the next exercise, where you'll use an RNN to build a character-level language model.